# Individual Assignment  - Q2. Sentiment Analysis via ML-Based Approach

In [230]:
# Name: Zita Lo
# Student Number: 20196119
# Program: MMA
# Cohort: Winter 2021
# Course Number: MMA 865
# Date: October 17, 2020


# Answer to Question 2 - Task 1

# Support response to Question 2 - Task 3: 
# ---- "Export Incorrect Prediction Results (sentiment_test data)" section
# ---- "Further exploration on Sentiment_Test Data" section 

In [231]:
# Import packages

import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [232]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.1.


In [ ]:
import os
os.getcwd()

# Load in data

In [234]:
# load in csv file. Display info about the data and show first 5 instances.

df = pd.read_csv("sentiment_train.csv")

# Set display width = 100.
pd.set_option('display.max_colwidth', 100)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB


,Sentence,Polarity
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
4,The selection on the menu was great and so were the prices.,1


In [235]:
# Split data into train and test set ratio 0.85 : 0.15.
# Define "Polarity" feature as the target variable.

from sklearn.model_selection import train_test_split

X = df['Sentence']
y = df['Polarity']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Preprocessing and Feature Engineering

In [236]:
# Preprocessing

# Import packages for preprocessing steps including removing stopwords, lemmatizing process, regular expression operations, etc.

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import unidecode
import textstat
import string  

# Define variable for WordNetLammatizer().
lemmer = WordNetLemmatizer()

# Define function for preprocessor. Input is a single document, as a single string.
# Output should be a single document, as a single string.

def my_preprocess(doc):
    
    # Lowercase
    doc = doc.lower()   
 
    # Substitute single characters with single space
    doc = re.sub(r'\s+[a-z]\s+', ' ', doc)
    
    # Substitute starting single characters with single space
    doc = re.sub(r'^[a-z]\s+', ' ', doc) 
    
    # Substitute digits with single space
    doc = re.sub('\d+', ' ', doc)
    
    # Substitute isn't, ain't, wasn't, didn't... with "not" to avoid "''" got removed in the cleaning process
    doc = re.sub(r"isn't","is not",doc)
    doc = re.sub(r"ain't","am not",doc)
    doc = re.sub(r"wasn't","was not",doc)
    doc = re.sub(r"didn't","did not",doc)
    doc = re.sub(r"don't","do not",doc)
    doc = re.sub(r"wouldn't","would not",doc)
    doc = re.sub(r"shouldn't","should not",doc)
    doc = re.sub(r"can't","can not",doc)
    doc = re.sub(r"couldn't","could not",doc)
    doc = re.sub(r"won't","will not",doc)
    
    # Substitute non word characters with single space
    doc = re.sub('\W+', ' ', doc)
    
    # Substitute multiple spaces with single space
    doc = re.sub(r'\s+', ' ', doc, flags=re.I)
    
    # Lemmatize each word
    doc = ' '.join([lemmer.lemmatize(w) for w in doc.split()])

    return doc

In [237]:
# Define functions for additional features in the document.
# They will later be put into the Pipeline and called via the FunctionTransformer() function.
# Each one takes an entier corpus (as a list of documents), and should return
# an array of feature values (one for each document in the corpus). 

# Import a few popular lexicon packages e.g. textblob, afinn, nltk.sentiment.vader.
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn

# Count length
def doc_length(corpus):
    return np.array([len(doc) for doc in corpus]).reshape(-1, 1)

# Count number of words present in the text
def lexicon_count(corpus):
    return np.array([textstat.lexicon_count(doc) for doc in corpus]).reshape(-1, 1)

# Get number of punctuation and sum them up
def _get_punc(doc):
    return len([a for a in doc if a in string.punctuation])

def punc_count(corpus):
    return np.array([_get_punc(doc) for doc in corpus]).reshape(-1, 1)

# Get number of upper case and sum them up
def _get_caps(doc):
    return sum([1 for a in doc if a.isupper()])

def capital_count(corpus):
    return np.array([_get_caps(doc) for doc in corpus]).reshape(-1, 1)

# Count number of exclamation marks
def num_exclamation_marks(corpus):
    return np.array([doc.count('!') for doc in corpus]).reshape(-1, 1)

# Count number of question marks
def num_question_marks(corpus):
    return np.array([doc.count('?') for doc in corpus]).reshape(-1, 1)

# Return boolean value if "not" exists or not
def has_not(corpus):
    return np.array([bool(re.search("not", doc.lower())) for doc in corpus]).reshape(-1, 1)
 
# Return sentiment polarity value from TextBlob lexicon
def sentiment_polar(corpus):
    return np.array([TextBlob(doc).sentiment.polarity for doc in corpus]).reshape(-1, 1)

# Return sentiment positive score value from nltk vader lexicon
def sid_pos(corpus):
    return np.array([SentimentIntensityAnalyzer().polarity_scores(doc)['pos'] for doc in corpus]).reshape(-1, 1) 

# Return sentiment compound score value from nltk vader lexicon
def sid_compound(corpus):
    return np.array([SentimentIntensityAnalyzer().polarity_scores(doc)['compound'] for doc in corpus]).reshape(-1, 1) 

# Return sentiment score value from Afinn lexicon
def afn(corpus):
    afn = Afinn(emoticons=True)
    return np.array([afn.score(doc) for doc in corpus]).reshape(-1, 1) 


In [238]:
# Calculate the class weights and check whether data has any class imbalance issue.
# Data turns out quite equaly distibuted for "Polarity".

import numpy as np
neg, pos = np.bincount(df['Polarity'])
total = neg + pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.99
Weight for class 1: 1.01


# Construct the Pipeline

In [239]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Need to preprocess the stopwords, because scikit learn's TfidfVectorizer
# removes stopwords _after_ preprocessing.
stop_words = [my_preprocess(word) for word in stop_words.ENGLISH_STOP_WORDS]

# This vectorizer will be used to create the BOW features.
vectorizer = TfidfVectorizer(preprocessor=my_preprocess, 
                             max_features = 1500, 
                             ngram_range=[1,2],
                             stop_words=[None],
                             strip_accents="unicode", 
                             lowercase=False, max_df=0.4, min_df=0.001, use_idf=True)

# This vectorizer will be used to preprocess the text before topic modeling.
vectorizer2 = TfidfVectorizer(preprocessor=my_preprocess, 
                             max_features = 1500, 
                             ngram_range=[1,4],
                             stop_words=[None],
                             strip_accents="unicode", 
                             lowercase=False, max_df=0.4, min_df=0.001, use_idf=True)

# Topic modelling NMF
nmf = NMF(n_components=25, random_state=1, init='nndsvda', solver='mu', alpha=.1, l1_ratio=.5)

# Algorithms - Random Forest (LR), Multi-layer Perceptron (MLP), ExtraTreeClassifier (ET), Logistic Regression (LR), and
# support vector machine (SVM)
rf = RandomForestClassifier(criterion='entropy', random_state=223)
mlp = MLPClassifier(random_state=42, verbose=2, max_iter=200)
et = ExtraTreesClassifier(random_state=101, n_estimators=200)
lr = LogisticRegression(random_state=101, max_iter=5000, C = 10, solver='liblinear')
svm = SVC(random_state=101,probability=True, kernel="linear", class_weight="balanced")

# Use FeatureUnion() to put all features together as preprocessing step.
feature_processing =  FeatureUnion([ 
    ('bow', Pipeline([('cv', vectorizer), ])),
    ('topics', Pipeline([('cv', vectorizer2), ('nmf', nmf),])),
    ('length', FunctionTransformer(doc_length, validate=False)),
    ('words', FunctionTransformer(lexicon_count, validate=False)),
    ('punc_count', FunctionTransformer(punc_count, validate=False)),
    ('capital_count', FunctionTransformer(capital_count, validate=False)),  
    ('num_exclamation_marks', FunctionTransformer(num_exclamation_marks, validate=False)),  
    ('num_question_marks', FunctionTransformer(num_question_marks, validate=False)),
    ('has_not', FunctionTransformer(has_not, validate=False)),    
    ('afn', FunctionTransformer(afn, validate=False)), 
    ('sentiment_polar', FunctionTransformer(sentiment_polar, validate=False)),    
    ('sid_pos', FunctionTransformer(sid_pos, validate=False)), 
    ('sid_compound', FunctionTransformer(sid_compound, validate=False)), 
])


# Use standard scaler to scale data
steps = [('features', feature_processing),('scaler', StandardScaler(with_std=True,with_mean=False))]



In [240]:
# Define the pipeline with feature processing, scaling, and them run model.
pipe = Pipeline([('features', feature_processing),('scaler', StandardScaler(with_std=True,with_mean=False)), ('clf', et)])

param_grid = {}

# ExtraTreeClassifier (ET) is chosen as the final optimal model. Currently "ET" is set as default under "which_clf" variable.
# To switch to other algorithm, simply changed the "which_clf" variable to "RF", "MLP", "LR" or "SVM".
# All these algorithms (ExtraTreeClassifier (ET), Random Forest (LR), Multi-layer Perceptron (MLP), Logistic Regression (LR), 
# support vector machine (SVM) have been tuned. The best parameters are listed and commented out in each section.

which_clf = "ET"

if which_clf == "RF":

    steps.append(('clf', rf))

    # Hypertuning for more than 3 hours (185.4 mins)
    #Best parameter (CV scy_train0.849):
    #{'clf__class_weight': None, 
    #'clf__criterion': 'entropy', 
    #'clf__n_estimators': 100, 
    #'features__bow__cv__max_features': 3000, 
    #'features__bow__cv__preprocessor': <function my_preprocess at 0x000001F725CB4D38>,
    #'features__bow__cv__use_idf': False, 
    #'features__topics__cv__stop_words': 'english',
    #'features__topics__nmf__n_components': 200}
    param_grid = {
        'features__bow__cv__preprocessor': [my_preprocess],
        'features__bow__cv__max_features': [1000,3000],
        'features__bow__cv__use_idf': [False, True],
        'features__topics__cv__stop_words': [None,'english'],
        'features__topics__nmf__n_components': [75,200,400],
        'clf__n_estimators': [100, 500],
        'clf__class_weight': [None],
        'clf__criterion': ['gini', 'entropy'],
    }
    
elif which_clf == "MLP":
    
    steps.append(('clf', mlp))
  
    #Best parameter (CV scy_train0.804):
    #{'clf__hidden_layer_sizes': (50, 50), 
    #'clf__solver': 'adam', 
    #'features__bow__cv__max_features': 3000, 
    #'features__bow__cv__min_df': 0, 
    #'features__bow__cv__preprocessor': <function my_preprocess at 0x000001BA2DA97D38>, 
    #'features__bow__cv__use_idf': False, 
    #'features__topics__cv__stop_words': 'english', 
    #'features__topics__nmf__n_components': 300}
    
    param_grid = {
        'features__bow__cv__preprocessor': [my_preprocess],
        'features__bow__cv__max_features': [3000],
        'features__bow__cv__min_df': [0],
        'features__bow__cv__use_idf': [False],
        'features__topics__cv__stop_words': ["english"],
        'features__topics__nmf__n_components': [300],
        'clf__solver': ["adam"],         
        'clf__hidden_layer_sizes': [(100, ), (50, 50)],
    }

elif which_clf == "ET":
    
    steps.append(('clf', et))
    
    #Best parameter (CV scy_train0.852):
    #{'clf__class_weight': None, 
    #'clf__criterion': 'entropy', 
    #'clf__n_estimators': 200, 
    #'features__bow__cv__max_features': 3000, 
    #'features__bow__cv__preprocessor': <function my_preprocess at 0x0000027D52AA1678>,
    #'features__bow__cv__use_idf': False,
    #'features__topics__cv__max_features': 1500,
    #'features__topics__cv__stop_words':stop_words,
    #'features__topics__nmf__n_components': 200
    
    param_grid = {
        'features__bow__cv__preprocessor': [my_preprocess],
        'features__bow__cv__max_features': [3000],
        'features__bow__cv__stop_words': [None, stop_words],
        'features__bow__cv__use_idf': [False],
        'features__topics__cv__max_features': [1500],
        'features__topics__cv__stop_words': [stop_words],
        'features__topics__nmf__n_components': [200],
        'clf__n_estimators': [200],
        'clf__class_weight': [None],   
        'clf__criterion': ['entropy'],
        
    }
    
elif which_clf == "LR":
    
    steps.append(('clf', lr))
    #Best parameter (CV scy_train0.839):
    #{'clf__C': 100, 'clf__solver': 'lbfgs', 
    #'features__bow__cv__max_features': 3000, 
    #'features__bow__cv__preprocessor': <function my_preprocess at 0x00000257B9123DC8>, 
    #'features__bow__cv__use_idf': False, 
    #'features__topics__cv__stop_words': stop_words,
    #'features__topics__nmf__n_components': 300
    param_grid = {
        'features__bow__cv__preprocessor': [my_preprocess],
        'features__bow__cv__max_features': [3000],
        'features__bow__cv__use_idf': [False, True],
        'features__topics__cv__stop_words': [stop_words],
        'features__topics__nmf__n_components': [300],        
        'clf__C': [1,10,100],         
        'clf__solver': ['lbfgs', 'liblinear'],
    }

elif which_clf == "SVM":
    
    steps.append(('clf', svm))    
    #Best parameter (CV scy_train0.829):
    #{'clf__class_weight': None, 
    #'clf__kernel': 'linear', 
    #'features__bow__cv__max_features': 3000, 
    #'features__bow__cv__preprocessor': <function my_preprocess at 0x000001BA2F0FC828>,
    #'features__bow__cv__use_idf': True, 
    #'features__topics__cv__stop_words': None, 
    #'features__topics__nmf__n_components': 400}   
 
    param_grid = {
        'features__bow__cv__preprocessor': [my_preprocess],
        'features__bow__cv__max_features': [3000],
        'features__bow__cv__use_idf': [True],
        'features__topics__cv__stop_words': [None],
        'features__topics__nmf__n_components': [400],
        'clf__kernel': ['linear'],
        'clf__class_weight': [None],
    }
    
pipe = Pipeline(steps)

# Use GridSearchCV() for cross validation and hyper-parameters tuning
search = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3, scoring='f1_micro', return_train_score=True, verbose=2)

# Fit Model

In [241]:
search = search.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:  2.6min finished


In [242]:
# Print best parameters

print("Best parameter (CV scy_train%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV scy_train0.852):
{'clf__class_weight': None, 'clf__criterion': 'entropy', 'clf__n_estimators': 200, 'features__bow__cv__max_features': 3000, 'features__bow__cv__preprocessor': <function my_preprocess at 0x0000021D3F1211F8>, 'features__bow__cv__stop_words': None, 'features__bow__cv__use_idf': False, 'features__topics__cv__max_features': 1500, 'features__topics__cv__stop_words': ['several', 'hereafter', 'the', 'nobody', 'neither', 'ever', 'somewhere', 'become', 'she', 'twelve', 'amoungst', 'also', 'now', 'should', 'therein', 'down', 'seems', 'must', 'cannot', 'per', 're', 'found', 'noone', 'of', 'too', 'whether', 'together', 'itself', 'my', 'every', 'any', 'fifty', 'seeming', 'already', 'ltd', 'wherein', 'everything', 'out', 'an', 'meanwhile', 'but', 'why', 'hereby', 'although', 'first', 'here', 'are', 'such', 'thereafter', 'anyhow', 'then', 'what', 'hasnt', 'never', 'they', 'nothing', 'whole', 'wherever', 'themselves', 'whoever', 'anyone', 'thin', 'becoming', 'may', '

In [243]:
# Print out the results of hyperparmater tuning

def cv_results_to_df(cv_results):
    results = pd.DataFrame(list(cv_results['params']))
    results['mean_fit_time'] = cv_results['mean_fit_time']
    results['mean_score_time'] = cv_results['mean_score_time']
    results['mean_train_score'] = cv_results['mean_train_score']
    results['std_train_score'] = cv_results['std_train_score']
    results['mean_test_score'] = cv_results['mean_test_score']
    results['std_test_score'] = cv_results['std_test_score']
    results['rank_test_score'] = cv_results['rank_test_score']

    results = results.sort_values(['mean_test_score'], ascending=False)
    return results

results = cv_results_to_df(search.cv_results_)
results

# Export to csv (comment out below line to allow export)
#results.to_csv('results2.csv', index=False)

,clf__class_weight,clf__criterion,clf__n_estimators,features__bow__cv__max_features,features__bow__cv__preprocessor,features__bow__cv__stop_words,features__bow__cv__use_idf,features__topics__cv__max_features,features__topics__cv__stop_words,features__topics__nmf__n_components,mean_fit_time,mean_score_time,mean_train_score,std_train_score,mean_test_score,std_test_score,rank_test_score
0,None,entropy,200,3000,<function my_preprocess at 0x0000021D3F1211F8>,None,False,1500,"[several, hereafter, the, nobody, neither, ever, somewhere, become, she, twelve, amoungst, also,...",200,28.856713,10.860554,0.99951,0.000693,0.851961,0.001834,1
1,None,entropy,200,3000,<function my_preprocess at 0x0000021D3F1211F8>,"[several, hereafter, the, nobody, neither, ever, somewhere, become, she, twelve, amoungst, also,...",False,1500,"[several, hereafter, the, nobody, neither, ever, somewhere, become, she, twelve, amoungst, also,...",200,36.657278,15.255821,0.99951,0.000693,0.847549,0.007995,2


# Estimate Model Performance

In [244]:
# Get references to the objects from the pipeline with the *best* hyperparameter settings,
# in order to explore those objects at a later time.


# The pipeline with the best performance.
pipeline = search.best_estimator_

# Get the feature processing pipeline, which can be used later.
feature_processing_obj = pipeline.named_steps['features']

# Find the vectorizer objects, NMF objects, and classifier objects.
pipevect= dict(pipeline.named_steps['features'].transformer_list)
vectorizer_obj = pipevect.get('bow').named_steps['cv']
vectorizer_obj2 = pipevect.get('topics').named_steps['cv']
nmf_obj = pipevect.get('topics').named_steps['nmf']
clf_obj = pipeline.named_steps['clf']

# Confirm vocabSize setting. Should match the output.
len(vectorizer_obj.get_feature_names())

1926

In [245]:
# Performance metrics of the model.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score

features_val = feature_processing_obj.transform(X_val).todense()

pred_val = search.predict(X_val)

print("Confusion matrix:")
print(confusion_matrix(y_val, pred_val))

print("\nF1 Score = {:.5f}".format(f1_score(y_val, pred_val, average='micro')))

print("\nClassification Report:")
print(classification_report(y_val, pred_val))

Confusion matrix:
[[176  16]
 [ 31 137]]

F1 Score = 0.86944

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.88       192
           1       0.90      0.82      0.85       168

    accuracy                           0.87       360
   macro avg       0.87      0.87      0.87       360
weighted avg       0.87      0.87      0.87       360



# Evaluate Performance on Test Data

In [246]:
# Performance metrics of the model on sentiment_test data.

# load in test data and predict.
test_df = pd.read_csv('sentiment_test.csv')

features_test = feature_processing_obj.transform(test_df['Sentence']).todense()
pred_test = search.predict(test_df['Sentence'])

# load in test data and define target vairable.
solutions_df = pd.read_csv('sentiment_test.csv')
y_test = solutions_df['Polarity']

print("Confusion matrix:")
print(confusion_matrix(y_test, pred_test))

print("\nF1 Score = {:.5f}".format(f1_score(y_test, pred_test, average="micro")))

print("\nClassification Report:")
print(classification_report(y_test, pred_test))

Confusion matrix:
[[263  24]
 [ 78 235]]

F1 Score = 0.83000

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.92      0.84       287
           1       0.91      0.75      0.82       313

    accuracy                           0.83       600
   macro avg       0.84      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



# Export Test Data Prediction Results

In [247]:
# Output the predictions to a csv file.

# Create a final data frame with three columns "Sentence", "Polarity", "Predicted"
df_test_pred = pd.DataFrame({'Sentence': test_df.Sentence, 'Polarity': test_df.Polarity, 'Predicted': pred_test})

# Create a new column "Validation" to compare the actual vs prediction
df_test_pred['Validation'] = ''

# loop through each instance to compare "Polarity" and "Predicted"
row = 0
for row in range(len(df_test_pred)):    
    if (df_test_pred['Polarity'][row] == df_test_pred['Predicted'][row]):
        result = 'Correct'
    else:
        result = 'Incorrect'
    df_test_pred['Validation'][row] = result
    row += 1

# View first 20 and last 20 rows of data frame. Set display width = 100.
pd.set_option('display.max_colwidth', 100)
df_test_pred.head(15)
df_test_pred.tail(15)

# Export to csv. Uncomment to create the file.    
#df_test_pred.to_csv("sentiment_test_predicted.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Sentence,Polarity,Predicted,Validation
0,A good commentary of today's love and undoubtedly a film worth seeing.,1,1,Correct
1,"For people who are first timers in film making, I think they did an excellent job!!",1,1,Correct
2,"It was very popular when I was in the cinema, a good house and very good reactions and plenty of...",1,1,Correct
3,It's a feel-good film and that's how I felt when I came out of the cinema!,1,1,Correct
4,It has northern humour and positive about the community it represents.,1,1,Correct
5,I rather enjoyed it.,1,1,Correct
6,I liked it.,1,1,Correct
7,I couldn't take them seriously.,0,0,Correct
8,It really created a unique feeling though.,1,1,Correct
9,Vivian Schilling did an excellent job with the script.,1,1,Correct


,Sentence,Polarity,Predicted,Validation
585,Still it's quite interesting and entertaining to follow.,1,1,Correct
586,;) Recommend with confidence!,1,1,Correct
587,This movie is well-balanced with comedy and drama and I thoroughly enjoyed myself.,1,1,Correct
588,It was a riot to see Hugo Weaving play a sex-obsessed gay real estate salesman who uses his clie...,1,0,Incorrect
589,":) Anyway, the plot flowed smoothly and the male-bonding scenes were a hoot.",1,1,Correct
590,"The opening sequence of this gem is a classic, and the cat n mouse games that follow are a delig...",1,1,Correct
591,Fans of the genre will be in heaven.,1,1,Correct
592,Lange had become a great actress.,1,1,Correct
593,It looked like a wonderful story.,1,1,Correct
594,I never walked out of a movie faster.,0,0,Correct


# Export Incorrect Prediction Results (sentiment_test data)

In [248]:
# Select the predictions that are incorrect.
# Export to a separate csv. This file will be used for responding on Q2 Task 3.

incorrect_pred = df_test_pred[df_test_pred.Validation == "Incorrect"]
incorrect_pred.head()

# Uncomment below to export to a separate csv.
# incorrect_pred.to_csv("incorrect_pred.csv")

,Sentence,Polarity,Predicted,Validation
12,Not too screamy not to masculine but just right.,1,0,Incorrect
36,"The soundtrack wasn't terrible, either.",1,0,Incorrect
38,"Still, it was the SETS that got a big ""10"" on my ""oy-vey"" scale.",1,0,Incorrect
43,The last 15 minutes of movie are also not bad as well.,1,0,Incorrect
48,My 8/10 score is mostly for the plot.,1,0,Incorrect


# Explore the Model Further

Understanding what the model learned.

## Print Topics

Print the top words for each of the NMF topics

In [249]:
n_top_words = 12
def get_top_words(H, feature_names):
    output = []
    for topic_idx, topic in enumerate(H):
        top_words = [(feature_names[i]) for i in topic.argsort()[:-n_top_words - 1:-1]]
        output.append(top_words)
        
    return pd.DataFrame(output) 

top_words = get_top_words(nmf_obj.components_, vectorizer_obj2.get_feature_names())
top_words

,0,1,2,3,4,5,6,7,8,9,10,11
0,great,great food,great product,great service,food great,director,choice,great price,great place,great film,working,really great
1,good,food good,good quality,good product,good food,good price,good service,stuff,really good,good thing,good time,good phone
2,service,customer,customer service,great service,fast,food service,friendly,service food,good service,unreliable,mobile,cooked
3,phone,new,phone ve,sturdy,ve,cell phone,cell,good phone,motorola,hand,phone great,nokia
4,place,recommend place,love place,phoenix,authentic,lot,great place,town,clean,place eat,impressive,hope
...,...,...,...,...,...,...,...,...,...,...,...,...
195,returning,changing,probably,phone,zero star,forget,funny,fun,fry,friendly staff,friendly,friend
196,liked,white,patio,movie,year ago,verizon,ago,delish,non,parent,choice,especially
197,script,direction,worse,problem,acting,bad,horrible,make,food average,form,fun,fry
198,kid,annoying,nasty,play,area,hilarious,watch,cover,girl,option,hit,parent


## Print Feature Importances



In [250]:
topic_feature_names = ["topic {}".format(i) for i in range(nmf_obj.n_components_)]

stat_feature_names = [t[0] for t in pipeline.named_steps['features'].transformer_list if t[0] not in ['topics', 'bow']]

feature_names = vectorizer_obj.get_feature_names() + topic_feature_names + stat_feature_names
len(feature_names)

feature_importances = None
if hasattr(clf_obj, 'feature_importances_'):
    feature_importances = clf_obj.feature_importances_

2137

In [251]:
features_train = feature_processing_obj.transform(X_train).todense()

if feature_importances is None:
    print("No Feature importances! Skipping.")
else:
    N = features_train.shape[1]

    ssum = np.zeros(N)
    avg = np.zeros(N)
    avg_pos = np.zeros(N)
    avg_neg = np.zeros(N)
    for i in range(N):
        ssum[i] = sum(features_train[:, i]).reshape(-1, 1)
        avg[i] = np.mean(features_train[:, i]).reshape(-1, 1)
        avg_pos[i] = np.mean(features_train[y_train==1, i]).reshape(-1, 1)
        avg_neg[i] = np.mean(features_train[y_train==0, i]).reshape(-1, 1)

    et = search.best_estimator_
    imp = pd.DataFrame(data={'feature': feature_names, 'imp': feature_importances, 'sum': ssum, 'avg': avg, 'avg_neg': avg_neg, 'avg_pos': avg_pos})
    imp = imp.sort_values(by='imp', ascending=False)
    imp.head(20)
    imp.tail(10)
    #imp.to_csv('importances.csv', index=False)

,feature,imp,sum,avg,avg_neg,avg_pos
2136,sid_compound,0.087217,300.616700,0.147361,-0.166900,0.462240
2134,sentiment_polar,0.061807,319.436854,0.156587,-0.109893,0.423590
2135,sid_pos,0.058776,398.151000,0.195172,0.058933,0.331678
2133,afn,0.043533,2237.000000,1.096569,-0.579824,2.776251
2132,has_not,0.019577,238.000000,0.116667,0.193928,0.039254
1023,not,0.017353,100.647117,0.049337,0.083068,0.015540
613,great,0.014197,56.951240,0.027917,0.000834,0.055054
1926,topic 0,0.010093,14.919174,0.007313,0.000261,0.014379
2127,words,0.006625,22250.000000,10.906863,11.387855,10.424926
2126,length,0.006148,121677.000000,59.645588,61.666014,57.621197


,feature,imp,sum,avg,avg_neg,avg_pos
62,and food,6.128079e-07,1.233394,0.000605,0.000000,0.001210
241,bunch of,5.764693e-07,0.728818,0.000357,0.000541,0.000173
214,biscuit,2.324298e-07,1.024377,0.000502,0.000231,0.000774
1445,terrific,2.085302e-07,0.822836,0.000403,0.000000,0.000807
780,is perfect,1.012414e-07,0.993901,0.000487,0.000000,0.000975
615,great deal,9.751969e-08,0.887990,0.000435,0.000000,0.000871
64,and friendly,9.143774e-08,0.856606,0.000420,0.000000,0.000841
1393,staff are,0.000000e+00,0.771312,0.000378,0.000188,0.000568
1992,topic 66,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1989,topic 63,0.000000e+00,0.000000,0.000000,0.000000,0.000000


# Further explanation on Sentiment_Train Data

Explain all predictions that were incorrect of a tree-based model.

Note: It will crash when using non tree-based model e.g., MLPClassifier.

In [252]:
if feature_importances is None:
    print("No Feature importances! Skipping.")
else:

    from treeinterpreter import treeinterpreter as ti

    prediction, bias, contributions = ti.predict(clf_obj, features_val)

    for i in range(len(features_val)):
        if y_val.iloc[i] == pred_val[i]:
            continue
        print("Instance {}".format(i))
        X_val.iloc[i]
        print("Bias (trainset mean) {}".format(bias[i]))
        print("Truth {}".format(y_val.iloc[i]))
        print("Prediction {}".format(prediction[i, :]))
        print("Feature contributions:")
        con = pd.DataFrame(data={'feature': feature_names, 
                                 'value': features_val[i].A1,
                                 'neg contr': contributions[i][:, 0],
                                 'pos contr': contributions[i][:, 1],
                                 'abs contr': abs(contributions[i][:, 1])})

        con = con.sort_values(by="abs contr", ascending=False)
        con['pos cumulative'] = con['pos contr'].cumsum() + bias[i][1]
        con.head(30)
        print("-"*20) 

Instance 1


'It fits so securely that the ear hook does not even need to be used and the sound is better directed through your ear canal.'

Bias (trainset mean) [0.5004902 0.4995098]
Truth 1
Prediction [0.445 0.555]
Feature contributions:


,feature,value,neg contr,pos contr,abs contr,pos cumulative
2133,afn,2.000000e+00,-0.122051,0.122051,0.122051,0.621561
2135,sid_pos,2.070000e-01,0.067199,-0.067199,0.067199,0.554362
2136,sid_compound,7.198000e-01,-0.048036,0.048036,0.048036,0.602398
2132,has_not,1.000000e+00,0.041185,-0.041185,0.041185,0.561213
2134,sentiment_polar,4.333333e-01,-0.015501,0.015501,0.015501,0.576715
1960,topic 34,2.798374e-01,0.008250,-0.008250,0.008250,0.568465
613,great,0.000000e+00,0.007952,-0.007952,0.007952,0.560513
1926,topic 0,3.170931e-75,0.007620,-0.007620,0.007620,0.552892
1023,not,1.767767e-01,-0.007077,0.007077,0.007077,0.559970
2127,words,2.500000e+01,0.006924,-0.006924,0.006924,0.553046


--------------------
Instance 25


...

--------------------
Instance 348


'Light weight, I hardly notice it is there.'

Bias (trainset mean) [0.5004902 0.4995098]
Truth 1
Prediction [0.8525 0.1475]
Feature contributions:


,feature,value,neg contr,pos contr,abs contr,pos cumulative
2135,sid_pos,0.000000,0.074499,-0.074499,0.074499,0.425011
2132,has_not,1.000000,0.052531,-0.052531,0.052531,0.372481
2134,sentiment_polar,0.054167,0.024441,-0.024441,0.024441,0.348039
2127,words,8.000000,0.020328,-0.020328,0.020328,0.327711
1023,not,0.000000,-0.018378,0.018378,0.018378,0.346090
2128,punc_count,2.000000,0.012584,-0.012584,0.012584,0.333506
2126,length,42.000000,0.012088,-0.012088,0.012088,0.321418
2008,topic 82,0.012117,-0.011730,0.011730,0.011730,0.333148
2136,sid_compound,0.000000,0.009099,-0.009099,0.009099,0.324049
613,great,0.000000,0.008442,-0.008442,0.008442,0.315607


--------------------


# Further exploration on Sentiment_Test Data
# Support Q2 Task 3 Response

Explain all predictions that were incorrect of a tree-based model.

Note: this only works on tree-based models, like RF, ET. This cell will crash when using, e.g., MLPClassifier



In [253]:
# Decompose the predictions into the bias term (which is just the testset mean) and individual feature contributions,
# in order to understand which features contributed to the difference and by how much.

if  feature_importances is None:
    print("No Feature importances! Skipping.")
else:

    from treeinterpreter import treeinterpreter as ti

    prediction, bias, contributions = ti.predict(clf_obj, features_test)

    for i in range(len(features_test)):
        if y_test[i] == pred_test[i]:
            continue
        print("Instance {}".format(i))
        test_df.iloc[i, :].Sentence
        print("Bias (testset mean) {}".format(bias[i]))
        print("Truth {}".format(y_test[i]))
        print("Prediction {}".format(prediction[i, :]))
        print("Feature contributions:")
        con = pd.DataFrame(data={'feature': feature_names,
                                 'value': features_test[i].A1,
                                 'neg contr': contributions[i][:, 0],
                                 'pos contr': contributions[i][:, 1],
                                 'abs contr': abs(contributions[i][:, 1])})
        con = con.sort_values(by="abs contr", ascending=False)
        con['pos cumulative'] = con['pos contr'].cumsum() + bias[i][1]
        con.head(30)
        print("-"*20) 

Instance 12


'Not too screamy not to masculine but just right.  '

Bias (testset mean) [0.5004902 0.4995098]
Truth 1
Prediction [0.8525 0.1475]
Feature contributions:


,feature,value,neg contr,pos contr,abs contr,pos cumulative
2135,sid_pos,0.000000e+00,0.070344,-0.070344,0.070344,0.429166
2132,has_not,1.000000e+00,0.043360,-0.043360,0.043360,0.385807
2127,words,9.000000e+00,0.023045,-0.023045,0.023045,0.362761
2126,length,5.000000e+01,0.012180,-0.012180,0.012180,0.350581
2134,sentiment_polar,1.928571e-01,0.010566,-0.010566,0.010566,0.340015
1023,not,6.324555e-01,0.008991,-0.008991,0.008991,0.331025
2128,punc_count,1.000000e+00,0.008668,-0.008668,0.008668,0.322357
2136,sid_compound,0.000000e+00,0.008134,-0.008134,0.008134,0.314223
613,great,0.000000e+00,0.008064,-0.008064,0.008064,0.306159
758,is,0.000000e+00,0.007379,-0.007379,0.007379,0.298780


--------------------
Instance 36


...

--------------------
Instance 588


"It was a riot to see Hugo Weaving play a sex-obsessed gay real estate salesman who uses his clients' houses for his trysts with the flaming Darren (Tom Hollander).  "

Bias (testset mean) [0.5004902 0.4995098]
Truth 1
Prediction [0.785 0.215]
Feature contributions:


,feature,value,neg contr,pos contr,abs contr,pos cumulative
2136,sid_compound,-2.960000e-01,0.092575,-0.092575,0.092575,0.406935
2135,sid_pos,7.700000e-02,0.078710,-0.078710,0.078710,0.328225
2132,has_not,0.000000e+00,-0.024417,0.024417,0.024417,0.352642
1023,not,0.000000e+00,-0.018537,0.018537,0.018537,0.371179
2126,length,1.650000e+02,0.016019,-0.016019,0.016019,0.355160
2134,sentiment_polar,3.083333e-01,-0.014354,0.014354,0.014354,0.369514
613,great,0.000000e+00,0.009529,-0.009529,0.009529,0.359985
51,and,0.000000e+00,0.008378,-0.008378,0.008378,0.351607
1926,topic 0,0.000000e+00,0.007768,-0.007768,0.007768,0.343840
2127,words,2.900000e+01,0.007286,-0.007286,0.007286,0.336554


--------------------
